# Imports

In [1]:
#imports
import os
import argparse
import ast
import pickle as pkl
from itertools import tee

import wandb
import numpy as np
import open3d as o3d

import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, GAE, LayerNorm, Linear
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_sparse import SparseTensor
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

INFO - 2023-10-14 10:07:45,587 - utils - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 2023-10-14 10:07:45,588 - utils - NumExpr defaulting to 8 threads.
INFO - 2023-10-14 10:07:48,042 - instantiator - Created a temporary directory at /tmp/tmpi7mftijk
INFO - 2023-10-14 10:07:48,043 - instantiator - Writing /tmp/tmpi7mftijk/_remote_module_non_scriptable.py


# Functions

In [5]:
############################################################################################

def get_data(path, organ, save=False):
    registered_mesh = []
    dirs = next(os.walk(path))[1]
    dataset = []
    dirs = dirs[:2]
    
    for dir in dirs:
        mesh = o3d.io.read_triangle_mesh(f'{path}{dir}/{organ}')
    
        vertices_data = np.asarray(mesh.vertices)
        triangles = np.asarray(mesh.triangles)
        vertices = torch.from_numpy(vertices_data).double()
        edges = []
        for triangle in triangles:
            edges.append([triangle[0], triangle[1]])
            edges.append([triangle[0], triangle[2]])
            edges.append([triangle[1], triangle[2]])
            
        edges_torch = [[],[]]
        edges =np.unique(np.array(edges), axis=0)
        for edge in edges:
            edges_torch[0].append(edge[0])
            edges_torch[1].append(edge[1])
    
        edges_torch = torch.from_numpy(np.asarray(edges_torch)).long()
    
        registered_mesh.append((vertices.type(torch.float32), edges_torch))
        data = Data(x=registered_mesh[0][0], y=registered_mesh[0][0].flatten(), edge_index=registered_mesh[0][1], num_nodes= len(registered_mesh[0][0]))
        dataset.append(data)
        # data = train_test_split_edges(data)
        # print(data)
    
    if(save):
        with open(f'../data/infograph/{organ}/data', 'wb') as f:
            pkl.dump(dataset, f)

    return dataset

############################################################################################

# Data

In [1]:

dataset = TUDataset("./Data/", name="MUTAG").shuffle()
dataset

MUTAG(188)

In [3]:
dataset[0]
#Data(x=registered_mesh[0][0], y=registered_mesh[0][1], edge_index=registered_mesh[0][1], num_nodes= len(registered_mesh[0][0]))

Data(edge_index=[2, 36], x=[17, 7], edge_attr=[36, 4], y=[1])

In [4]:

dataloader = DataLoader(dataset, batch_size=9)
for x in dataloader:
    print(x.num_graphs)
    break

9


/u/home/wyo/.conda/envs/digital_twin/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
dataset = get_data("../../../../../../../vol/aimspace/users/wyo/registered_meshes/2000/", "liver_mesh.ply")
dataset[0]

Data(x=[1087, 3], edge_index=[2, 4555], y=[3261], num_nodes=1087)

In [6]:
test_ids_path = "../../../data/NonNa_organs_split_test.txt"
path = "../../../../../../../vol/aimspace/users/wyo/registered_meshes/2000/"
dirs = next(os.walk(path))[1]
test_dirs = np.loadtxt(test_ids_path, delimiter=",", dtype=str)

print(dirs[0], test_dirs[0])


3166348 2809772


In [2]:
import torch
from main import InfoGraph
model = torch.load('../../../models/liver_mesh.ply_infograph.pt', map_location=torch.device('cpu'))

In [3]:
from embed import get_single_subject
graph = get_single_subject("../../../../../../../vol/aimspace/users/wyo/registered_meshes/2000/", "liver_mesh.ply", str(1000071))

In [5]:
y, M = model.embed(graph.x, graph.edge_index) 
print(y.shape, M.shape)

torch.Size([1, 320]) torch.Size([1087, 320])
